In [ ]:
import math
import scipy
from math import *
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import rayleigh
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.integrate as integrate
from scipy.interpolate import interp1d
from mayavi import mlab
import plotly.graph_objects as go

In [ ]:
# Find the power of 2 that is greater than the number you give it
def nextPowerOf2(i):
    n = 1
    while n < i: n *= 2
    return n

def alpha_Thorp(f):
    # EJB: shameless copy from MAA
    # absorption formula from Thorpe: https://www.researchgate.net/publication/224599959_Variability_of_available_capacity_due_to_the_effects_of_depth_and_temperature_in_the_underwater_acoustic_communication_channel
    # f = frequency(kHz)
    # output in dB/km 

    # Boron relaxation frequency
    F1 = 1.0

    # Magnesium relaxation frequency
    F2 = sqrt(4100.)

    A1 = 0.1/0.9144*((f**2)/(f**2 + F1**2))
    A2 = (40/0.9144)*((f**2)/(f**2 + F2**2))
    A3 = 2.75e-4* f**2
    A3 = A3/0.9144

    return A1+A2+A3

# The following function provides a comprehensive assessment of high-frequency wind-induced noise levels in underwater environments
def NL_wind_highfreq(freqs, v_wind, depth):
    # APL from Ainslie (2010)
    dT = 1                                                                               # Time difference parameter, typically set to 1    
    deltaT = 0.26*(dT-1)**2                                                              # Temporal coherence parameter for wind-induced noise
    
    # Represents the contribution of wind speed to the noise spectral density
    # Describes how wind speed affects noise level at different frequencies
    Kwind = 10**(4.12)*v_wind**(2.24) / ((freqs/1000)**(1.59)*10**(0.1*deltaT))          # Wind-induced noise spectral density coefficient

    # from Ward et al. 2011                                              
    beta = alpha_Thorp(freqs/1000)/4343                                                  # Coefficient related to Thorp's attenuation coefficient
    
    # Accounts for the attenuation of noise with depth underwater
    # Reflects the decrease in noise level with increasing water depth
    depth_correction = -alpha_Thorp(freqs/1000) * (depth/1000) - 10*np.log10(1+beta*depth/2)    # Correction term accounting for water depth
    
    
    # Represents the combined effect of wind-induced noise and depth-related 
    # attenuation on the noise level at different frequencies underwater.
    return 10*np.log10(Kwind) + depth_correction                                            # spectral density [dB re 1 muPa^2/Hz]

def NL_wind_highfreq(freqs, v_wind, depth):
    return 1

# Neutrino

In [ ]:
def correct_offaxis_neutrino(theta, freq):
    # distance from the source to the detector, assumed to be 2 units
    d = 2
    
    # wavelength of the underwater signal which is speed of sound divided by the frequency
    wl = 1500./freq
    
    # initial intensity of the signal (assumed to be 1 unit)
    I_0 = 1.
    
    # below the formula to find the intensity of a signal under different angles is used
    if theta == 0:
        return I_0
    else:
        return I_0 * pow(sin((pi*d/wl)*sin(theta))/((pi*d/wl)*sin(theta)), 2)

In [ ]:
# Neutrino propegation
def get_neutrino_sourcewaveform():
    """
    Generates a bipolar pulse with appropriate amp and period. 
    The function runs as bipolar_pulse(amp, pulse_width) where amp is in uPa, pulse_width is in seconds. 
    """
    
    fs = 500e3                          # sampling rate
    t = np.arange(0, 100e-5, 1/fs)      # time range to generate the pulse over 
    t = t-np.mean(t)
    p_norm = 0.06                       # Pa
    Pa2uPa = 1e6                        # conversion factor to micropascal
    amp = p_norm*Pa2uPa                 # microPa
    pulse_width = 1e-4                  # seconds
    
    # expression to generate a bipolar wave
    y = -1 * amp * (t / (0.1*pulse_width)) * np.exp(-((t/(0.1*pulse_width))**2 - 1) / 2)

    # SPL (Sound Pressure Level): SPL is a measure of the sound pressure level in decibels (dB) relative to the reference 
    # sound pressure of 1 microPascal (µPa). It is calculated as the logarithm of the ratio of the root mean square (RMS) 
    # pressure of the waveform to the reference pressure, multiplied by 20. In the code, SPL is calculated using the formula 
    # (y represents the waveform):
    SPL = 20 * np.log10(np.sqrt(np.mean(y**2)))
    
    # E_t represents the energy level of the waveform and is also expressed in decibels (dB). It is calculated as the 
    # logarithm of the total energy of the waveform over its duration, normalized to one second, and then multiplied by 10. 
    # In the code, E_t is calculated using the formula:
    E_t = 10 * np.log10(np.sum(y**2)/fs)

    print(f'SPL = {SPL} dB re 1uPa and SEL = {E_t} dB re 1uPa')
    
    # returns the time array, the waveform, and the sampling frequency
    return t, y, fs

t, y, fs = get_neutrino_sourcewaveform()
plt.figure(figsize=(13, 10))

plt.plot(t, y/1000, color='dodgerblue')
plt.xlabel('time [s]')
plt.ylabel('amplitude [mPa]')
plt.show()

In [ ]:
def sound_level_Neutrino():
    """
    Generates the sound level (SL) distribution over angle for a neutrino
    """
    ###########################################################################################################################
    # set up the conditions to calculate sound level for the sperm whale and neutrino 
    ###########################################################################################################################        
    # angle w.r.t. axis of click [rad]
    thetas = np.linspace(-90, 90, 360)
    SL_thetas = []

    ###########################################################################################################################
    # Calculate the sound level for the neutrino in a similar manner
    ########################################################################################################################### 
    
    # Generate an array of frequencies from 5000 to 50000
    f_arr = np.linspace(5000, 50000, 100)
    I_theta = np.zeros(len(thetas))

    # Loop over the frequencies and calculate the intensity of the neutrino at different angles
    for freq in f_arr:
        I_theta += np.asarray(list(map(correct_offaxis_neutrino, np.radians(thetas), [freq]*len(thetas))))
    
    # calculate the average intensity of the signal over different frequencies
    I_theta_mean = np.asarray(I_theta) / len(f_arr)
    
    # get the sound level relative to the original neutrino sound in log scale
    SL_thetas = [10*math.log10(x) for x in I_theta_mean]
    
    # from all the values in the list substract the maximum values such that all identities are negative and 0 is the maximum
    SL_thetas -= np.max(SL_thetas)      
    
    return SL_thetas

In [ ]:
def propagate_neutrino():
    """
    Generates a bipolar pulse with appropriate amp and period. 
    The function runs as bipolar_pulse(amp, pulse_width) where amp is in uPa, pulse_width is in seconds. 
    """
    
    # get the original neutrino waveform
    ttt, p_t, fs = get_neutrino_sourcewaveform()
    
    ###########################################################################################################################
    # DEFINE FREQUENCY DOMAIN AND POWER SPECTRAL DENSITY OF THE SIGNAL
    ###########################################################################################################################

    # The code calculates the next power of 2 for the length of the waveform 
    nfft = nextPowerOf2(len(ttt))
    
    # Pad the waveform with zeros for FFT calculations
    p_t_e = np.pad(p_t, (0, nfft - len(p_t)), mode='constant')
    
    # Apply fft to the padded waveform to obtain the frequency domain representation
    X_f = np.fft.fft(p_t_e)
    
    # Calculate frequency resolution
    df = fs / nfft 
    
    # Generate frequency array corresponding to the FFT output
    freqs = np.arange(0, fs/2, df)

    # Slice the FFT output to keep only positive frequencies
    X_f = X_f[:len(freqs)]

    # Calculate the normalized power spectral density
    X_f2 = abs(X_f)**2 * (2 / (nfft * fs))
          
    # The calculated ESL represents the average energy level of the waveform over its entire duration
    ESL = 10*np.log10(sum(p_t**2)/fs)   # in dB broadband energy source level
    
    # The calculated EPSDL represents the energy level distribution across different frequency components of the signal
    EPSDL = 10*np.log10(X_f2)        # in dB source energy spectral density level
    
    # make the frequency array in kHz
    freqs_kHz = freqs / 1000
    
    # plot the results
    plt.plot(freqs, EPSDL, color='dodgerblue')
    plt.title('EPSDL')
    plt.ylabel('PSD')
    #plt.xscale('log')
    plt.xlabel('Frequency [Hz]')
    plt.show()
    plt.clf()

    ###########################################################################################################################
    # DEFINE THE SOUND LEVEL AT DIFFERENT OUTGOING ANGLES
    ###########################################################################################################################

    # Wind speed
    v_wind = 9                       # m/s 0 = SS0; 2 = SS2; 3.5 = SS3; 6-9 = SS 4; 
    depth = 1000                     # receiver depth

    NLwind = NL_wind_highfreq(freqs, v_wind, depth) + 10*log10(1e-4)
 
    range_neutrino = 1000            # m
    
    # calculate the power loss of the neutrino through the water
    # this term accounts for the spreading loss of the signal as it propagates through space: -20*log10(range_neutrino)
    # this term represents the absorption loss of the signal due to the medium, which is typically water in underwater acoustic scenarios: alpha_Thorp(freqs/1000) * (range_neutrino/1000)
    PL = -20*log10(range_neutrino) - alpha_Thorp(freqs/1000) * (range_neutrino/1000)
    
    # generate the SL for both the neutrino and the whale
    SL_Neutrino = sound_level_Neutrino()
    
    ###########################################################################################################################
    # SET UP THE ENVIRONMENT FOR PLOTTING    
    ###########################################################################################################################

    # define a meshgrid
    xxx = np.linspace(0, 10000, 500)
    yyy = np.linspace(0, 10000, 500)
    zzz = np.linspace(0, 2000, 100)
    
    x_arr, y_arr, z_arr = np.meshgrid(xxx, yyy, zzz)
    
    # set the source location
    x_src = 500
    y_src = 5000
    z_src = 1000

    # set the source location
    x_src_plot = [x_src]
    y_src_plot = [y_src]
    z_src_plot = [z_src]

    """    
    # create a figure
    fig = go.Figure()

    # add a scatter3d trace for the point
    fig.add_trace(go.Scatter3d( x=z_src_plot, y=y_src_plot, z=x_src_plot, mode='markers', marker=dict(size=10, color='dodgerblue')))

    # set layout options
    fig.update_layout(
        scene=dict(
            xaxis=dict(title='X Axis', range=[0, 10000]),     # Set x-axis range
            yaxis=dict(title='Y Axis', range=[0, 10000]),     # Set y-axis range
            zaxis=dict(title='Depth [m]', range=[0, 2000])))  # Set z-axis range

    # show the plot
    fig.show()
    """    

    ###########################################################################################################################
    # calculate the sound level solely on angle  
    ###########################################################################################################################        
    thetas = np.linspace(-90, 90, 360)
           
    # grid point calculations
    # first calculate the distance between the point in the grid and the source location of the neutrino
    r_arr = np.sqrt((x_arr - x_src)**2 + (y_arr - y_src)**2 + (z_arr - z_src)**2)
    # then calculate the angle between the point in the grid and the source location
    angle_r = np.rad2deg(np.arctan((z_src-z_arr) / (np.sqrt((y_arr - y_src)**2 + (x_arr - x_src)**2))))

    # SL_corr = interp1(theta, 10*np.log10(I_theta_mean), angle_r)
    set_interp = interp1d(thetas, SL_Neutrino, kind='linear')
    SL_corr = set_interp(angle_r)
    

    """
    # Create the 3D scatter plot
    fig = go.Figure(data=go.Scatter3d(
        x=z_arr.flatten(),  # Flatten the z_arr array to create a 1D array of x coordinates
        y=y_arr.flatten(),  # Flatten the y_arr array to create a 1D array of y coordinates
        z=x_arr.flatten(),  # Flatten the x_arr array to create a 1D array of z coordinates
        mode='markers',
        marker=dict(
            size=5,                         # Adjust the size of the markers as needed
            color=SL_corr.flatten(),        # Use SL_corr values for color
            colorscale='Viridis',           # Choose a colormap for the colors
            opacity=0.8,                    # Adjust the opacity of the markers
            colorbar=dict(title='SL_corr')  # Add a colorbar with title
        )
    ))

    # Add the source location as a red point
    fig.add_trace(go.Scatter3d(
        x=[z_src],
        y=[y_src],
        z=[x_src],
        mode='markers',
        marker=dict(
            size=10,
            color='dodgerblue'
        ),
        name='Source Location'
    ))

    # Set axis labels and plot title
    fig.update_layout(
        scene=dict(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            zaxis=dict(title='Z'),
        ),
        title='SL_corr around Source Location'
    )
    # Show the plot
    fig.show()
    """    
    
    ###########################################################################################################################
    # calculate the final sound level  
    ###########################################################################################################################        
     
    # calculated Sound Exposure Level (SEL):
    # this term accounts for contribution from the EPSDl: 10*log10(np.sum(10**((EPSDL)/10)*df))
    # this term represents the SL correction factor: SL_corr 
    # this term accounts for the spreading loss of sound as it propagates through water: 20*np.log10(r_arr)
    # This term represents the absorption loss of sound : 10*log10(np.mean(10**(alpha_Thorp(freqs/1000)/10)))*r_arr/1000
    sel_rec = 10*log10(np.sum(10**((EPSDL)/10)*df)) + SL_corr - 20*np.log10(r_arr) - 10*log10(np.mean(10**(alpha_Thorp(freqs/1000)/10)))*r_arr/1000      

    return sel_rec, SL_corr, angle_r

In [ ]:
sel_rec_n, SL_corr_n, angle_r_n = propagate_neutrino()

print(sel_rec_n.shape)
print(SL_corr_n.shape)
print(angle_r_n.shape)

## Figure

In [ ]:
# set the threshold range around -150
threshold_range = 3  # You can adjust this value as needed

# filter points where sel_rec is within the threshold range around -70
sel_rec_filtered = np.where(np.abs(sel_rec_n + 150) < threshold_range, sel_rec_n, np.nan)
non_nan_indices = np.logical_not(np.isnan(sel_rec_filtered))

# check the amount of points with NaN values in them
if non_nan_indices.any():
    print("Array contains non-NaN values.")
    print(non_nan_indices.any())

num_non_nan = np.count_nonzero(non_nan_indices)
print("Number of non-NaN values:", num_non_nan)

# extract the corresponding x, y, z values using the non-NaN indices
x_non_nan = x_arr[non_nan_indices]
y_non_nan = y_arr[non_nan_indices]
z_non_nan = z_arr[non_nan_indices]

# make the other half of the signal
z_non_nan_half = abs(z_non_nan - 1000) + 1000

# create the 3D surface plot
fig = go.Figure()

# Add the first mesh surface
fig.add_trace(go.Mesh3d(
    x=x_non_nan,
    y=y_non_nan,
    z=z_non_nan,
    opacity=0.2,
    color='gold',
    name='Surface 1'
))

# Add the second mesh surface
fig.add_trace(go.Mesh3d(
    x=x_non_nan,
    y=y_non_nan,
    z=z_non_nan_half,
    opacity=0.2,
    color='gold',
    name='Surface 2'
))

# Add the source location as a red point
fig.add_trace(go.Scatter3d(
    x=[x_src],
    y=[y_src],
    z=[z_src],
    mode='markers',
    marker=dict(
        size=5,
        color='dodgerblue',
    ),
    name='Source Location'
))

aspect_ratio = dict(x=1, y=1, z=0.2)

fig.update_layout(
    scene=dict(
        xaxis=dict(title='X', range=[0, 10000]),   # Set x-axis limits
        yaxis=dict(title='Y', range=[0, 10000]),   # Set y-axis limits
        zaxis=dict(title='Z', range=[0, 2000]),    # Set z-axis limits
        aspectmode="manual",                       # Set aspect mode to manual
        aspectratio=aspect_ratio                   # Set custom aspect ratio 
    ),
    title='sel_rec around Source Location'
)

# Show the plot
fig.show()

# Sperm Whale

correction factor for peak pressure for off-axis clicks, as proposed in Zimmer et al. 2005. Apply this correcion factor to peak value of click (not energy source level)!

Some variables:
* a_piston = size of piston, used to represent the size of the sound generation part of the animal [m]
* theta_offaxis = angle w.r.t. axis of click [rad]
* c_w = speed of sound in water [m/s]
* freqs = frequenty range to consider [Hz], will integrate from min to max
* frequency specified in this array
* f_0_weight = central frequency [Hz] of Gaussian function fitted to click
* power spectrum
* b_weight = SE [Hz] of Gaussian form fitted to click power spectrum


$function SL_offaxis = correct_offaxis_SL(a_piston, theta_offaxis, c_w, freqs, f_0_weight, b_weight)$

*Source: Zimmer, W. M. X., Johnson, M. P., Madsen, P. T., and Tyack, P. L. (2005). Echolocation clicks of free-ranging Cuviers beaked whales ( Ziphius cavirostris), J. Acoust. Soc. Am. 117, 3919-3927.*


#### method:
Assume piston model to model frequency dependent beampattern and integrate over all frequencies to obtain broadband beampattern.

$P(x) = P_0 * (2 * J_1(x)/ x)$

with:
* P_0 = source level (NOTE: should be source factor for the peak sound pressure)
* J_1 = first order Bessel function
* x = k*a*sin(theta) = 2*pi*a* sin(theta)/c_w * f

broadband beampattern

$B(theta) = \frac{\int_{-\infty}^{\infty} { P^2 (\theta, f) * W^2(f) df }}{\int_{-\infty}^{\infty} {W^2(f) df}}$

with:
* $W^2 = exp(\frac{-1/2*(f-f_0)^2}{b^2})$
* b and f_0 obtained by least-squere fit to measured power spectrum
* $DI = 10*log10(\frac{B(0)*\int_0^{\pi}{sin(\theta)*d\theta}}{\int_{0}^{\pi}{sin(\theta)d\theta}})$
* the -3 dB beamwidth is approximately 185 deg x $10^{-DI/20}$
* P_0 = 1; normalize to one
* $x = \frac{2*\pi*a\_piston*sin(theta_offaxis)}{c_w * freqs}$
* P_x = P_0*(2*besselj(1,x)./x);
* fun = @(x,c) 1./(x.^3-2*x-c);
* Evaluate the integral from x=0 to x=2 at c=5.
* q = integral(@(x)fun(x,5),0,2)

In [ ]:
# eq 4 Zimmer et al
def P(f, theta_offaxis, a_piston, f_0_weight, b_weight, c_w):
        P_0 = 1.
        xxx = 2*math.pi*a_piston*math.sin(theta_offaxis)/c_w * f
        P_x = P_0*(2*scipy.special.j1(xxx)/xxx)
        
        return P_x**2*W(f, f_0_weight, b_weight) **2

def W(f, f_0_weight, b_weight):
        # eq 6 Zimmer et al
        return math.exp(-1/2*(f - f_0_weight)**2/ b_weight**2)

In [ ]:
def sound_level_Whale():
    """
    Generates the sound level (SL) distribution over angle for both a neutrino and a sperm whale
    """
    ###########################################################################################################################
    # set up the conditions to calculate sound level for the sperm whale and neutrino 
    ###########################################################################################################################        
    # angle w.r.t. axis of click [rad]
    thetas = np.linspace(-90, 90, 360)
    SL_offaxis = []
    SL_thetas = []

    # size of piston, used to represent the size of the sound generation part of the animal [m]
    a_piston = 0.16                           
    
    c_w = 1500                                   # speed of sound in water [m/s]
    freqs = np.linspace(24000, 50000, 1000)      # frequenty range to consider [Hz], will integrate from min to max frequency specified in this array
    f_0_weight = 38300                           # central frequency [Hz] of Gaussian function fitted to click
    b_weight = 6900                              # SE [Hz] of Gaussian form fitted to click power spectrum
    fmin = min(freqs)
    fmax = max(freqs)

    ###########################################################################################################################
    # Calculate the sound level with the help of the functions P and W above for sperm whale
    ###########################################################################################################################        
    
    for itheta in [math.radians(x) for x in thetas]:
        if itheta == 0:
            SL_offaxis.append(0)
        else:
            # eq 5 Zimmer et al
            B_theta = integrate.quad(P, fmin, fmax, args=(itheta, a_piston, f_0_weight, b_weight, c_w))[0] \
                      /integrate.quad(W, fmin, fmax, args=(f_0_weight, b_weight))[0]
            SL_offaxis.append(10*math.log10(B_theta))
    SL_offaxis -= np.max(SL_offaxis) 
    
    return SL_offaxis

In [ ]:
def propagate_sperm_whale():
    """
    Generates a signal similar to one a sperm whale would create
    """

    ###########################################################################################################################
    # DEFINE THE SOUND LEVEL AT DIFFERENT OUTGOING ANGLES
    ###########################################################################################################################

    SL_Sperm_Whale = sound_level_Whale()
        
    # Sperm Whale
    thetas = np.linspace(-90, 90, 360)
    
    # calculate the average intensity of the signal over different frequencies
    I_theta_mean_sperm = [10 ** (x / 10) for x in SL_Sperm_Whale]
    
    ###########################################################################################################################
    # SET UP THE ENVIRONMENT FOR PLOTTING    
    ###########################################################################################################################

    # define a meshgrid
    xxx = np.linspace(0, 10000, 500)
    yyy = np.linspace(0, 10000, 500)
    zzz = np.linspace(0, 2000, 100)
    
    x_arr, y_arr, z_arr = np.meshgrid(xxx, yyy, zzz)
    
    # set the source location
    x_src = 500
    y_src = 5000
    z_src = 1000

    ###########################################################################################################################
    # calculate the sound level solely on angle  
    ###########################################################################################################################        
  
    # grid point calculations
    # first calculate the distance between the point in the grid and the source location of the neutrino
    r_arr = np.sqrt((x_arr - x_src)**2 + (y_arr - y_src)**2 + (z_arr - z_src)**2)
    # then calculate the angle between the point in the grid and the source location
    angle_r = np.rad2deg(np.arctan((x_src-x_arr) / (np.sqrt((y_arr - y_src)**2 + (z_arr - z_src)**2))))
    
    # make the angle point in the right direction
    def transform_angle(angle):
        return np.where(angle < 0, -90 - angle, 90 - angle)

    # Apply transformation to angle_r
    angle_r = transform_angle(angle_r)
    
    # SL_corr = interp1(theta, 10*np.log10(I_theta_mean), angle_r)
    set_interp = interp1d(thetas, SL_Sperm_Whale, kind='linear')
    SL_corr = set_interp(angle_r)

    ###########################################################################################################################
    # calculate the final sound level  
    ###########################################################################################################################        
     
    # calculated Sound Exposure Level (SEL):
    # this term represents the SL correction factor: SL_corr_beam 
    # this term accounts for the spreading loss of sound as it propagates through water: 20*np.log10(r_arr)
    # This term represents the absorption loss of sound : 10*log10(np.mean(10**(alpha_Thorp(freqs/1000)/10)))*r_arr/1000
    sel_rec = SL_corr - 20*np.log10(r_arr) - alpha_Thorp(35) * r_arr / 1000

    return sel_rec, SL_corr, angle_r

In [ ]:
sel_rec, SL_corr, angle_r = propagate_sperm_whale()

print(sel_rec.shape)
print(SL_corr.shape)
print(angle_r.shape)

## Checks
### angle

In [ ]:
print(np.max(angle_r))
print(np.min(angle_r))

abs_angle_r = abs(angle_r)

print(np.min(abs_angle_r))

min_100_indices = np.argpartition(angle_r.flatten(), 1000)[:1000]
max_100_indices = np.argpartition(angle_r.flatten(), -1000)[-1000:]
zero_100_indices = np.argpartition(abs_angle_r.flatten(), 1000)[:1000]

# Extract coordinates of the top 100 points
x_min_100 = x_arr.flatten()[min_100_indices]
y_min_100 = y_arr.flatten()[min_100_indices]
z_min_100 = z_arr.flatten()[min_100_indices]
x_max_100 = x_arr.flatten()[max_100_indices]
y_max_100 = y_arr.flatten()[max_100_indices]
z_max_100 = z_arr.flatten()[max_100_indices]
x_zero_100 = x_arr.flatten()[zero_100_indices]
y_zero_100 = y_arr.flatten()[zero_100_indices]
z_zero_100 = z_arr.flatten()[zero_100_indices]

# Print coordinates of the top 100 points
#print("Coordinates and angles of the top 100 points:")
#for i in range(100):
    #index = top_100_indices[i]
    #print(f"Point {i+1}: x={x_top_100[i]}, y={y_top_100[i]}, z={z_top_100[i]}, angle={angle_r.flatten()[index]}")
    
# Create a trace for the 3D scatter plot
trace_min = go.Scatter3d(
    x=x_min_100,
    y=y_min_100,
    z=z_min_100,
    mode='markers',
    marker=dict(
        size=5,
        color='gold',  # You can adjust the color as needed
        opacity=0.05
    ),
    name='-90 degrees'
)

# Create a trace for the 3D scatter plot
trace_max = go.Scatter3d(
    x=x_max_100,
    y=y_max_100,
    z=z_max_100,
    mode='markers',
    marker=dict(
        size=5,
        color='blue',  
        opacity=0.05
    ),
    name='+90 degrees'
)
    
# Create a trace for the 3D scatter plot
trace_zero = go.Scatter3d(
    x=x_zero_100,
    y=y_zero_100,
    z=z_zero_100,
    mode='markers',
    marker=dict(
        size=5,
        color='orange',  
        opacity=0.05
    ),
    name='0 degrees'
)

# Create a trace for the single red point
trace_single_point = go.Scatter3d(
    x=[500],
    y=[5000],
    z=[1000],
    mode='markers',
    marker=dict(
        size=5,
        color='dodgerblue', 
        opacity=1
    ),
    name='Source Location'
)

aspect_ratio = dict(x=1, y=1, z=0.2)

# Create layout for the plot
layout = go.Layout(
    scene=dict(
        xaxis=dict(title='X', range=[0, 10000]),   # Set x-axis limits
        yaxis=dict(title='Y', range=[0, 10000]),   # Set y-axis limits
        zaxis=dict(title='Z', range=[0, 2000]),    # Set z-axis limits
        aspectmode="manual",                       # Set aspect mode to manual
        aspectratio=aspect_ratio                   # Set custom aspect ratio 
    ),
    title='angle_r'
)

# Create Figure object and add both traces
fig = go.Figure(data=[trace_zero, trace_max, trace_min, trace_single_point], layout=layout)

# Show the plot
fig.show()

### SL_corr

In [ ]:
print(f"min: {np.min(SL_corr)}")
print(f"max: {np.max(SL_corr)}")

max_indices = np.unravel_index(np.argmin(SL_corr), SL_corr.shape)

print("Indices of the maximum value in the entire array:", max_indices)

# Assume max_indices contains the indices of the maximum value in the array
x_max = x_arr[max_indices]
y_max = y_arr[max_indices]
z_max = z_arr[max_indices]

print("Coordinates of the point with the maximum value:")
print("x:", x_max)
print("y:", y_max)
print("z:", z_max)

print("-----------------------------------------")

# find the points with the maximum and minimum values
min_100_indices = np.argpartition(SL_corr.flatten(), 1000)[:1000]
max_100_indices = np.argpartition(SL_corr.flatten(), -1000)[-1000:]

# extract coordinates of the top 100 points
x_min_100 = x_arr.flatten()[min_100_indices]
y_min_100 = y_arr.flatten()[min_100_indices]
z_min_100 = z_arr.flatten()[min_100_indices]
x_max_100 = x_arr.flatten()[max_100_indices]
y_max_100 = y_arr.flatten()[max_100_indices]
z_max_100 = z_arr.flatten()[max_100_indices]

# print coordinates of the top 100 points
#print("Coordinates of the top 100 points:")
#for i in range(100):
    #print(f"Point {i+1}: x={x_top_100[i]}, y={y_top_100[i]}, z={z_top_100[i]}")
    
# create a trace for the 3D scatter plot
trace_min = go.Scatter3d(
    x=x_min_100,
    y=y_min_100,
    z=z_min_100,
    mode='markers',
    marker=dict(
        size=5,
        color='gold',  
        opacity=0.05
    ),
    name='min'
)

# create a trace for the 3D scatter plot
trace_max = go.Scatter3d(
    x=x_max_100,
    y=y_max_100,
    z=z_max_100,
    mode='markers',
    marker=dict(
        size=5,
        color='blue', 
        opacity=0.05
    ),
    name='max'
)
# Create a trace for the single red point
trace_single_point = go.Scatter3d(
    x=[500],
    y=[5000],
    z=[1000],
    mode='markers',
    marker=dict(
        size=5,
        color='dodgerblue',
        opacity=1
    ),
    name='Source Location'
)

aspect_ratio = dict(x=1, y=1, z=0.2)

# Create layout for the plot
layout = go.Layout(
    scene=dict(
        xaxis=dict(title='X', range=[0, 10000]),   # Set x-axis limits
        yaxis=dict(title='Y', range=[0, 10000]),   # Set y-axis limits
        zaxis=dict(title='Z', range=[0, 2000]),    # Set z-axis limits
        aspectmode="manual",                       # Set aspect mode to manual
        aspectratio=aspect_ratio                   # Set custom aspect ratio 
    ),
    title='SL_corr'
)

# Create Figure object and add both traces
fig = go.Figure(data=[trace_min, trace_max, trace_single_point], layout=layout)

# Show the plot
fig.show()

### sel_rec

In [ ]:
print(f"min: {np.min(sel_rec)}")
print(f"max: {np.max(sel_rec)}")

max_indices = np.unravel_index(np.argmin(sel_rec), sel_rec.shape)

print("Indices of the maximum value in the entire array:", max_indices)

# Assume max_indices contains the indices of the maximum value in the array
x_max = x_arr[max_indices]
y_max = y_arr[max_indices]
z_max = z_arr[max_indices]

print("Coordinates of the point with the maximum value:")
print("x:", x_max)
print("y:", y_max)
print("z:", z_max)

print("-----------------------------------------")

# find the points with the maximum and minimum values
min_100_indices = np.argpartition(sel_rec.flatten(), 1000)[:1000]
max_100_indices = np.argpartition(sel_rec.flatten(), -100000)[-100000:]

# extract coordinates of the top 100 points
x_min_100 = x_arr.flatten()[min_100_indices]
y_min_100 = y_arr.flatten()[min_100_indices]
z_min_100 = z_arr.flatten()[min_100_indices]
x_max_100 = x_arr.flatten()[max_100_indices]
y_max_100 = y_arr.flatten()[max_100_indices]
z_max_100 = z_arr.flatten()[max_100_indices]

# extract the sel_rec values corresponding to max_100_indices
sel_rec_max_100 = sel_rec.flatten()[max_100_indices]

# find the index of the minimum value in sel_rec_max_100
min_sel_rec_index = np.argmin(sel_rec_max_100)

# extract the minimum sel_rec value
min_sel_rec_value = sel_rec_max_100[min_sel_rec_index]

print("Minimum sel_rec value in max_100_indices:", min_sel_rec_value)
print("-----------------------------------------")

# print coordinates of the top 100 points
#print("Coordinates of the top 100 points:")
#for i in range(100):
    #print(f"Point {i+1}: x={x_top_100[i]}, y={y_top_100[i]}, z={z_top_100[i]}")
    
# create a trace for the 3D scatter plot
trace_min = go.Scatter3d(
    x=x_min_100,
    y=y_min_100,
    z=z_min_100,
    mode='markers',
    marker=dict(
        size=5,
        color='gold',  
        opacity=0.05
    ),
    name='min'
)

# create a trace for the 3D scatter plot
trace_max = go.Scatter3d(
    x=x_max_100,
    y=y_max_100,
    z=z_max_100,
    mode='markers',
    marker=dict(
        size=5,
        color='blue', 
        opacity=0.05
    ),
    name='max'
)
# Create a trace for the single red point
trace_single_point = go.Scatter3d(
    x=[500],
    y=[5000],
    z=[1000],
    mode='markers',
    marker=dict(
        size=5,
        color='dodgerblue',
        opacity=1
    ),
    name='Source Location'
)

aspect_ratio = dict(x=1, y=1, z=0.2)

# Create layout for the plot
layout = go.Layout(
    scene=dict(
        xaxis=dict(title='X', range=[0, 10000]),   # Set x-axis limits
        yaxis=dict(title='Y', range=[0, 10000]),   # Set y-axis limits
        zaxis=dict(title='Z', range=[0, 2000]),    # Set z-axis limits
        aspectmode="manual",                       # Set aspect mode to manual
        aspectratio=aspect_ratio                   # Set custom aspect ratio 
    ),
    title='SL_corr'
)

# Create Figure object and add both traces
fig = go.Figure(data=[trace_min, trace_max, trace_single_point], layout=layout)

# Show the plot
fig.show()

In [ ]:
# set the threshold range around -150
threshold_range = 3  # You can adjust this value as needed

# filter points where sel_rec is within the threshold range around -70
sel_rec_filtered = np.where(np.abs(sel_rec + 100) < threshold_range, sel_rec, np.nan)
non_nan_indices = np.logical_not(np.isnan(sel_rec_filtered))

# check the amount of points with NaN values in them
if non_nan_indices.any():
    print("Array contains non-NaN values.")
    print(non_nan_indices.any())

num_non_nan = np.count_nonzero(non_nan_indices)
print("Number of non-NaN values:", num_non_nan)

# extract the corresponding x, y, z values using the non-NaN indices
x_non_nan = x_arr[non_nan_indices]
y_non_nan = y_arr[non_nan_indices]
z_non_nan = z_arr[non_nan_indices]

# make the other half of the signal
z_non_nan_half = abs(z_non_nan - 1000) + 1000

# create the 3D surface plot
fig = go.Figure()

# Add the first mesh surface
fig.add_trace(go.Mesh3d(
    x=x_non_nan,
    y=y_non_nan,
    z=z_non_nan,
    opacity=0.2,
    color='gold',
    name='Surface 1'
))

# Add the second mesh surface
fig.add_trace(go.Mesh3d(
    x=x_non_nan,
    y=y_non_nan,
    z=z_non_nan_half,
    opacity=0.2,
    color='gold',
    name='Surface 2'
))

# Add the source location as a red point
fig.add_trace(go.Scatter3d(
    x=[x_src],
    y=[y_src],
    z=[z_src],
    mode='markers',
    marker=dict(
        size=5,
        color='dodgerblue',
    ),
    name='Source Location'
))

aspect_ratio = dict(x=1, y=1, z=0.2)

fig.update_layout(
    scene=dict(
        xaxis=dict(title='X', range=[0, 10000]),   # Set x-axis limits
        yaxis=dict(title='Y', range=[0, 10000]),   # Set y-axis limits
        zaxis=dict(title='Z', range=[0, 2000]),    # Set z-axis limits
        aspectmode="manual",                       # Set aspect mode to manual
        aspectratio=aspect_ratio                   # Set custom aspect ratio 
    ),
    title='sel_rec around Source Location'
)

# Show the plot
fig.write_image("3d_surface_plot_whale.png", width=1200, height=800, scale=2)
fig.show()

# Noise

In [ ]:
# Define theta values
theta = np.arange(0, 181, 1) / 180 * np.pi

# Calculate Heaviside functions
heavi_theta = np.zeros(len(theta))
sel = np.where(theta < 0)
heavi_theta[sel] = 0
sel = np.where(theta > 0)
heavi_theta[sel] = 1
sel = np.where(theta == 0)
heavi_theta[sel] = 0.5

heavi_theta_halfpi = np.zeros(len(theta))
sel = np.where(theta - np.pi / 2 < 0)
heavi_theta_halfpi[sel] = 0
sel = np.where(theta - np.pi / 2 > 0)
heavi_theta_halfpi[sel] = 1
sel = np.where(theta - np.pi / 2 == 0)
heavi_theta_halfpi[sel] = 0.5

# Calculate F_theta
F_theta = (heavi_theta - heavi_theta_halfpi) * 4 * np.cos(theta)

###########################################################################################################################
# Plot the results 
###########################################################################################################################        

# Plot polar pattern
fig = plt.figure()
ax = fig.add_subplot(111, polar=True)
ax.plot(theta, np.maximum(0, 10 * np.log10(F_theta) + 40), 'k-')

# set the theta zero location (0 degrees at the top)
ax.set_theta_zero_location('W')

# add labels and title
plt.xlabel('Angle [degrees]')
plt.ylabel('radiated noise level [dB]')
plt.title('Noise Intensity compared to Sea Surface (0 degrees)')
    
plt.show()



# Plot F_theta vs theta
plt.figure()
plt.plot(theta/pi*180, 10*np.log10(F_theta),'k')
plt.ylim([-40, 10])
plt.xlim([0, 180])
plt.xlabel('Angle [degrees]')
plt.ylabel('Radiated Noise Level [dB]')
plt.show()

# Define frequency and depth
fff = 10000  # (5000:1000:50000)
depth = 500

# Calculate F_theta2
F_theta2 = 3 / (2 * np.pi) * np.sin(theta) * np.exp(-2 * alpha_Thorp(fff / 1000) / (20 * np.log10(np.exp(1))) * (depth / 1000) / np.sin(theta))
print(F_theta2)
print(np.log10(F_theta2))

# Plot F_theta2
plt.figure()
plt.plot(theta/pi*180-90, 10*np.log10(F_theta2),'k--')
plt.ylim([-40, 10])
plt.xlim([0, 180])
plt.xlabel('Angle [degrees]')
plt.ylabel('Radiated Noise Level [dB]')
plt.show()

# Plot F_theta2 with different depths
plt.figure()
plt.subplot(2, 1, 1)
plt.plot(theta/pi*180, 10*np.log10(F_theta)-max(10*np.log10(F_theta)),'k')
plt.ylim([-40, 10])
plt.xlim([0, 180])
plt.ylabel('Radiated Noise Level [dB]')

ls = ['-', '--', ':', '.-']
fff_arr = [5000, 10000, 25000, 50000]
for j in range(4):
    fff = fff_arr[j]
    depth = 2000
    F_theta2 = 3 / (2 * np.pi) * np.sin(theta) * np.exp(-2 * alpha_Thorp(fff / 1000) / (20 * np.log10(np.exp(1))) * (depth / 1000) / np.sin(theta))
    plt.plot(theta/pi*180-90, 10*np.log10(F_theta2)-max(10*np.log10(F_theta2)), 'k' + ls[j])

plt.legend(['5 kHz', '10 kHz', '25 kHz', '50 kHz'])
plt.subplot(2, 1, 2)
plt.plot(theta/pi*180, 10*np.log10(F_theta)-max(10*np.log10(F_theta)), 'k')
plt.ylim([-40, 10])
plt.xlim([0, 180])
plt.ylabel('Radiated Noise Level [dB]')
plt.xlabel('Angle [degrees]')

depth_arr = [500, 1000, 2000, 3000]
for j in range(4):
    fff = 10000
    depth = depth_arr[j]
    F_theta2 = 3 / (2 * np.pi) * np.sin(theta) * np.exp(-2 * alpha_Thorp(fff / 1000) / (20 * np.log10(np.exp(1))) * (depth / 1000) / np.sin(theta))
    plt.plot(theta/pi*180-90, 10*np.log10(F_theta2)-max(10*np.log10(F_theta2)), 'k' + ls[j])

plt.legend(['500 m', '1000 m', '2000 m', '3000 m'])
plt.show()

# Calculate bottom contribution
fff = 15000  # (5000:1000:50000)
depth = 500
depth_bottom = 4000
F_theta2 = 3 / (2 * np.pi) * np.sin(theta) * np.exp(-2 * alpha_Thorp(fff / 1000) / (20 * np.log10(np.exp(1))) * (depth / 1000) / np.sin(theta))

F_theta2_bottom = (3 / (2 * np.pi) * np.sin(theta) * np.exp(-2 * alpha_Thorp(fff / 1000) / (20 * np.log10(np.exp(1))) * ((depth_bottom + (depth_bottom - depth)) / 1000) / np.sin(theta)))

F_theta2_tot = np.maximum(F_theta2, F_theta2_bottom)

# Plot bottom contribution
plt.figure()
plt.plot(theta/pi*180-90, 10*np.log10(F_theta2),'k--')
plt.plot(theta/pi*180+90, 10*np.log10(F_theta2_bottom),'k--')
plt.xlabel('Angle [degrees]')
plt.ylabel('Radiated Noise Level [dB]')
plt.ylim([-40, 10])
plt.xlim([0, 180])
plt.show()